# [Mechanisms of Action (MoA) Prediction](https://www.kaggle.com/c/lish-moa)

### I use the notebook [Pytorch CV|0.0145| LB| 0.01839 |](https://www.kaggle.com/riadalmadani/pytorch-cv-0-0145-lb-0-01839) from [riadalmadani](https://www.kaggle.com/riadalmadani) as a basis and will try to tune its various parameters. 

# Acknowledgements

* [MoA | Pytorch | 0.01859 | RankGauss | PCA | NN](https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn)
* [[MoA] Pytorch NN+PCA+RankGauss](https://www.kaggle.com/nayuts/moa-pytorch-nn-pca-rankgauss)
* [Pytorch CV|0.0145| LB| 0.01839 |](https://www.kaggle.com/riadalmadani/pytorch-cv-0-0145-lb-0-01839)
* [[New Baseline] Pytorch | MoA](https://www.kaggle.com/namanj27/new-baseline-pytorch-moa)
* [Deciding (n_components) in PCA](https://www.kaggle.com/kushal1506/deciding-n-components-in-pca)
* tuning and visualization from [Higher LB score by tuning mloss - upgrade & visual](https://www.kaggle.com/vbmokin/higher-lb-score-by-tuning-mloss-upgrade-visual)

### My upgrade:

* PCA parameters
* VarianceThreshold in the Feature Selection by Variance Encoding
* Dropout
* Structuring of the notebook
* Tuning visualization

I used the code from sources (please see above). But I am planning to develop this notebook. There are still promising areas for improvement and research of parameters.

## 1. Import libraries<a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

In [1]:
import sys
sys.path.append('../input/iterativestratification')

import numpy as np
import random
import pandas as pd
import os
import copy
import gc

import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import warnings
warnings.filterwarnings('ignore')

os.listdir('../input/lish-moa')

pd.set_option('max_columns', 2000)

### Radu upgrade:
* Remove ctl_vehicle at first
* MultiLabel Stratification by Drug (Chris Deotte)
* Cluster Features

In [2]:

extra_features_=True
add_clusters_= True
drop_c62 = True
n_comp_GENES = int(0.6*772)
n_comp_CELLS = int(0.6*100)
VarianceThreshold_for_FS = 0.9
Dropout_Model = 0.25
print('n_comp_GENES', n_comp_GENES, 'n_comp_CELLS', n_comp_CELLS, 'total', n_comp_GENES + n_comp_CELLS)

n_comp_GENES 463 n_comp_CELLS 60 total 523


## 3. Download data<a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')

#drop
#test_features = test_features.append(train_features, ignore_index=True).sample(frac=1).reset_index(drop=True)

sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
print(train_features.shape)
print(test_features.shape)

(23814, 876)
(3982, 876)


In [5]:
#Remove ctl_vehicle at first
train_features = train_features[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test_features = test_features[test_features['cp_type']!='ctl_vehicle']
train_targets_scored = train_targets_scored[ train_targets_scored['sig_id'].isin(train_features['sig_id']) ].reset_index(drop=True)

In [6]:
if drop_c62:
    train_features = train_features.drop(['c-62'],axis=1)
    test_features = test_features.drop(['c-62'],axis=1)

In [7]:
print(train_features.shape)
print(train_targets_scored.shape)
print(test_features.shape)

(21948, 875)
(21948, 207)
(3624, 875)


In [8]:
train_features

sig_id cp_type  cp_time cp_dose     g-0     g-1     g-2     g-3  \
0      id_000644bb2  trt_cp       24      D1  1.0620  0.5577 -0.2479 -0.6208   
1      id_000779bfc  trt_cp       72      D1  0.0743  0.4087  0.2991  0.0604   
2      id_000a6266a  trt_cp       48      D1  0.6280  0.5817  1.5540 -0.0764   
3      id_0015fd391  trt_cp       48      D1 -0.5138 -0.2491 -0.2656  0.5288   
4      id_001626bd3  trt_cp       72      D2 -0.3254 -0.4009  0.9700  0.6919   
...             ...     ...      ...     ...     ...     ...     ...     ...   
21943  id_fff8c2444  trt_cp       72      D1  0.1608 -1.0500  0.2551 -0.2239   
21944  id_fffb1ceed  trt_cp       24      D2  0.1394 -0.0636 -0.1112 -0.5080   
21945  id_fffb70c0c  trt_cp       24      D2 -1.3260  0.3478 -0.3743  0.9905   
21946  id_fffcb9e7c  trt_cp       24      D1  0.6660  0.2324  0.4392  0.2044   
21947  id_ffffdd77b  trt_cp       72      D1 -0.8598  1.0240 -0.1361  0.7952   

          g-4     g-5     g-6     g-7     g-8     g-9    g-10    g-11    g-12  \
0     -0.1944 -1.0120 -1.0220 -0.0326  0.5548 -0.0921  1.1830  0.1530  0.5574   
1      1.0190  0.5207  0.2341  0.3372 -0.4047  0.8507 -1.1520 -0.4201 -0.0958   
2     -0.0323  1.2390  0.1715  0.2155  0.0065  1.2300 -0.4797 -0.5631 -0.0366   
3      4.0620 -0.8095 -1.9590  0.1792 -0.1321 -1.0600 -0.8269 -0.3584 -0.8511   
4      1.4180 -0.8244 -0.2800 -0.1498 -0.8789  0.8630 -0.2219 -0.5121 -0.9577   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
21943 -0.2431  0.4256 -0.1166 -0.1777 -0.7480  0.1368  0.5493 -0.5083  0.4112   
21944 -0.4713  0.7201  0.5773  0.3055 -0.4726  0.1269  0.2531  0.1730 -0.4532   
21945 -0.7178  0.6621 -0.2252 -0.5565  0.5112  0.6727 -0.1851  2.8650 -0.2140   
21946  0.8531 -0.0343  0.0323  0.0463  0.4299 -0.7985  0.5742  0.1421  2.2700   
21947 -0.3611 -3.6750 -1.2420  0.9146  3.0790  1.2460  1.9460  1.4370  2.9780   

         g-13    g-14    g-15    g-16    g-17    g-18    g-19    g-20    g-21  \
0     -0.4015  0.1789 -0.6528 -0.7969  0.6342  0.1778 -0.3694 -0.5688 -1.1360   
1      0.4590  0.0803  0.2250  0.5293  0.2839 -0.3494  0.2883  0.9449 -0.1646   
2     -1.8300  0.6057 -0.3278  0.6042 -0.3075 -0.1147 -0.0570 -0.0799 -0.8181   
3     -0.5844 -2.5690  0.8183 -0.0532 -0.8554  0.1160 -2.3520  2.1200 -1.1580   
4      1.1750  0.2042  0.1970  0.1244 -1.7090 -0.3543 -0.5160 -0.3330 -0.2685   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
21943  0.1181  0.7861 -0.1877  0.2476  0.1956  0.0273  0.1182 -0.5037 -0.5939   
21944 -1.0790  0.2474 -0.4550  0.3588  0.1600 -0.7362 -0.1103  0.8550 -0.4139   
21945 -0.6153  0.8362  0.5584 -0.2589  0.1292  0.0148  0.0949 -0.2182 -0.9235   
21946  0.2046  0.5363 -1.7330  0.1450  0.6097  0.2024  0.9865 -0.7805  0.9608   
21947  2.2370 -0.6818  0.6870 -1.1060  0.0182 -0.9247 -0.0738 -0.1919 -0.7722   

         g-22    g-23    g-24    g-25    g-26    g-27    g-28    g-29    g-30  \
0     -1.1880  0.6940  0.4393  0.2664  0.1907  0.1628 -0.2853  0.5819  0.2934   
1     -0.2657 -0.3372  0.3135 -0.4316  0.4773  0.2075 -0.4216 -0.1161 -0.0499   
2     -1.5320  0.2307  0.4901  0.4780 -1.3970  4.6240 -0.0437  1.2870 -1.8530   
3     -0.7191 -0.8004 -1.4670 -0.0107 -0.8995  0.2406 -0.2479 -1.0890 -0.7575   
4      0.7649  0.2057  1.3720  0.6835  0.8056 -0.3754 -1.2090  0.2965 -0.0712   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
21943  0.4825 -0.5187  0.1618 -0.2253  0.3153  0.4839  0.0520  1.3850  0.6502   
21944  0.5541  0.2310 -0.5573 -0.4397 -0.9260 -0.2424 -0.6686  0.2326  0.6456   
21945  0.0749 -1.5910 -0.8359 -0.9217  0.3013  0.1716  0.0880  0.1842  0.1835   
21946  0.3440  2.7650  0.4925  0.6698  0.2374 -0.3372  0.8771 -2.6560 -0.2000   
21947 -1.4050 -1.0050 -1.1170 -0.5293 -1.1720 -0.2885  0.1599 -0.4250  0.3591   

         g-31    g-32    g-33    g-34    g-35    g-36    g-37    g-38    g-39  \
0     -0.5584 -0.0916 -0.3010 -0.1537  0.2198  0.296

## 4. FE & Data Preprocessing <a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

In [9]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
col_cat = ['cp_time', 'cp_dose']

### 4.1 RankGauss<a class="anchor" id="4.1"></a>

[Back to Table of Contents](#0.1)

In [10]:
# RankGauss - transform to Gauss

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [11]:
train_features[(GENES + CELLS)].head(10)

g-0       g-1       g-2       g-3       g-4       g-5       g-6  \
0  1.111801  0.903367 -0.433829 -0.971728 -0.286559 -1.011388 -1.357431   
1  0.105667  0.672509  0.257486  0.086759  1.199685  0.691813  0.353695   
2  0.767036  0.942499  1.408911 -0.126492 -0.028694  1.490985  0.272541   
3 -0.755626 -0.297077 -0.455058  0.765972  2.343522 -0.852713 -2.316440   
4 -0.468806 -0.504196  0.956769  0.975864  1.447729 -0.863807 -0.346926   
5 -0.903768  0.497868 -1.435580  0.342624  1.375307  0.691355  0.116503   
6  1.468032  2.146911 -1.850048  2.473610 -2.331979 -2.204814  2.428145   
7  0.362227  0.835585 -0.283303  2.055754  0.926221  0.769060 -0.325189   
8 -0.431900  0.898399 -0.445438 -0.224426 -0.973598 -0.673020 -1.006730   
9 -0.090205  0.442237 -0.820545 -0.409224 -0.008370 -1.333672 -0.265456   

        g-7       g-8       g-9      g-10      g-11      g-12      g-13  \
0 -0.041716  0.719019 -0.298798  1.546151  0.155864  0.661714 -0.548724   
1  0.558374 -0.497516  0.845472 -1.243593 -0.590105 -0.181974  0.547967   
2  0.359490  0.038331  1.246733 -0.651185 -0.782581 -0.098601 -2.292704   
3  0.301512 -0.150669 -1.358486 -1.008581 -0.509950 -1.117638 -0.802450   
4 -0.227072 -1.026456  0.858798 -0.344280 -0.715015 -1.216237  1.171604   
5 -0.336401 -0.184374  0.345204 -0.012615  0.402509  0.944049 -1.273527   
6  2.172751  1.157207 -1.841454  2.477055 -2.702454  2.402430  2.138671   
7 -1.616234  0.642548  1.476863 -0.123535 -0.107909  0.170326  0.217512   
8  0.153996 -0.101293  1.410021  0.744699  0.009495  0.453419  0.888833   
9 -0.058624  0.014748  0.635390 -0.270427 -0.927836  0.843403 -1.132216   

       g-14      g-15      g-16      g-17      g-18      g-19      g-20  \
0  0.295737 -1.071349 -1.120205  0.904383  0.363779 -0.523377 -0.698297   
1  0.165179  0.383917  0.413317  0.431386 -0.855132  0.489923  1.269330   
2  0.888461 -0.558999  0.485378 -0.379193 -0.321293 -0.054187 -0.076751   
3 -1.739990  1.385318 -0.209308 -1.029882  0.229093 -2.129180  2.080983   
4  0.333034  0.335085 -0.009513 -1.537557 -0.866434 -0.720440 -0.422244   
5  1.096327 -0.429178  0.578208  0.103088 -1.054728  1.755627  1.567621   
6  1.844563  2.493738 -1.934530 -1.179911  2.504974 -0.784195 -2.197059   
7 -0.376744 -0.317329  0.450641  1.715645  1.551087 -1.235980  1.091982   
8 -1.209789  0.768495 -0.812293  0.642816  0.096708 -0.514376 -0.929025   
9  0.537384  0.411652  0.989285 -0.558129  1.403268 -0.309041 -0.159371   

       g-21      g-22      g-23      g-24      g-25      g-26      g-27  \
0 -1.369550 -1.695536  1.250671  0.570589  0.436027  0.290501  0.129888   
1 -0.149157 -0.423792 -0.458679  0.402412 -0.555557  0.689364  0.186370   
2 -1.045070 -1.951027  0.533785  0.640179  0.735237 -1.349676  2.391083   
3 -1.390113 -1.126062 -1.073452 -1.405924  0.034704 -1.023687  0.231648   
4 -0.309671  1.018179  0.493778  1.709450  1.042281  1.140661 -0.607406   
5  0.568648 -0.818810 -0.382494 -1.253160 -0.402097 -0.148737  1.088841   
6  1.680457  1.476298  0.843390  0.440897  2.490110  2.551262 -2.019743   
7 -0.357820  2.134393 -0.806679 -1.132731  1.106594  0.246229 -2.246373   
8  0.690468 -0.700623  0.114185 -0.873169 -0.458958  0.887077  0.783198   
9  0.620711 -1.080423  0.546174 -0.967392 -0.989827  1.685064  1.156264   

       g-28      g-29      g-30      g-31      g-32      g-33      g-34  \
0 -0.481883  0.761154  0.419666 -1.109027 -0.059113 -0.409266 -0.235059   
1 -0.709338 -0.208375 -0.120770 -0.552012  1.673802 -0.336817  0.319165   
2 -0.048482  1.576074 -1.423752  0.806211  0.739079  0.240407 -0.036651   
3 -0.415427 -1.250653 -0.965690  0.126443 -2.062941  1.078408  0.748901   
4 -1.509471  0.364082 -0.153460  0.836779  1.140934 -0.105424  1.461888   
5  1.339953  0.001142  0.433540 -1.734006  0.917784  1.327267  1.418147   
6  2.507607  2.479729  2.546003  1.661947  1.558485  1.772393  2.151723   
7  1.568467 -1.757824  0.278251  1.081290 -0.200579  0.304884  1.302351   
8 -0.584778 -1.041834  0.6097

In [12]:

def create_cluster(train, test, features, kind = 'g', n_clusters = 35):
    train_ = train[features].copy()
    test_ = test[features].copy()
    data = pd.concat([train_, test_], axis = 0)
    kmeans = KMeans(n_clusters = n_clusters, random_state = 62).fit(data)
    train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
    test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
    #train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
    #test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
    return train, test
    
def transform_data(train, test, col, features_g, features_c, removed_vehicle=False, extra_features=False, add_clusters=False):
    """
        the first 3 columns represents categories, the others numericals features
    """
    mapping = {"cp_type":{"trt_cp": 0, "ctl_vehicle":1},
               "cp_time":{48:0, 72:1, 24:2},
               "cp_dose":{"D1":0, "D2":1}}
    
    if extra_features:

        for df in train, test:
            df['g_sum'] = df[features_g].sum(axis = 1)
            df['g_mean'] = df[features_g].mean(axis = 1)
            df['g_std'] = df[features_g].std(axis = 1)
            df['g_kurt'] = df[features_g].kurtosis(axis = 1)
            df['g_skew'] = df[features_g].skew(axis = 1)
            df['c_sum'] = df[features_c].sum(axis = 1)
            df['c_mean'] = df[features_c].mean(axis = 1)
            df['c_std'] = df[features_c].std(axis = 1)
            df['c_kurt'] = df[features_c].kurtosis(axis = 1)
            df['c_skew'] = df[features_c].skew(axis = 1)
            df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
            df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
            df['gc_std'] = df[features_g + features_c].std(axis = 1)
            df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
            df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
    if add_clusters:
    
        train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = 35)
        train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = 5)
        
    categories_tr = np.stack([ train[c].apply(lambda x: mapping[c][x]).values for c in col], axis=1)
    categories_test = np.stack([ test[c].apply(lambda x: mapping[c][x]).values for c in col], axis=1)
        
    if add_clusters:
        categories_tr = np.append(categories_tr, train[['clusters_g', 'clusters_c']].values, axis=1)
        categories_test = np.append(categories_test, test[['clusters_g', 'clusters_c']].values, axis=1)                               
   
    if extra_features:
        numerical_tr = train[(GENES + CELLS + ['g_sum', 'g_mean', 'g_std', 'g_kurt', 'g_skew', 'c_sum', 'c_mean', 'c_std', 'c_kurt', 'c_skew', 'gc_sum', 'gc_mean', 'gc_std', 'gc_kurt', 'gc_skew'])].values
        numerical_test = test[(GENES + CELLS + ['g_sum', 'g_mean', 'g_std', 'g_kurt', 'g_skew', 'c_sum', 'c_mean', 'c_std', 'c_kurt', 'c_skew', 'gc_sum', 'gc_mean', 'gc_std', 'gc_kurt', 'gc_skew'])].values
    else:
        numerical_tr = train[(GENES + CELLS)].values
        numerical_test = test[(GENES + CELLS)].values

    return categories_tr, categories_test, numerical_tr, numerical_test
    
cat_tr, cat_test, numerical_tr, numerical_test = transform_data(train_features, test_features, col_cat, GENES, CELLS, extra_features=extra_features_, add_clusters=add_clusters_)

In [13]:
print(numerical_tr.shape)
print(cat_tr.shape)
print(len(GENES + CELLS))

(21948, 886)
(21948, 4)
871


In [14]:
887 + 4 + n_comp_GENES + n_comp_CELLS

1414

### 4.2 Seed<a class="anchor" id="4.2"></a>

[Back to Table of Contents](#0.1)

In [15]:
def seed_everything(seed=62):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=62)

### 4.3 PCA features<a class="anchor" id="4.3"></a>

[Back to Table of Contents](#0.1)

In [16]:
len(GENES)

772

In [17]:
# GENES

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp_GENES, random_state=62).fit_transform(data[GENES]))
PCA_tr_g = data2[:train_features.shape[0]]; 
PCA_test_g = data2[-test_features.shape[0]:]
'''
train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)'''

"\ntrain2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])\ntest2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])\ntrain_features = pd.concat((train_features, train2), axis=1)\ntest_features = pd.concat((test_features, test2), axis=1)"

In [18]:
len(CELLS)

99

In [19]:
# CELLS

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp_CELLS, random_state=62).fit_transform(data[CELLS]))
PCA_tr_c = data2[:train_features.shape[0]]; 
PCA_test_c = data2[-test_features.shape[0]:]
'''
train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)'''

"\ntrain2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])\ntest2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])\ntrain_features = pd.concat((train_features, train2), axis=1)\ntest_features = pd.concat((test_features, test2), axis=1)"

In [20]:
numerical_tr2 = np.concatenate([numerical_tr, PCA_tr_g, PCA_tr_c], axis=1)
numerical_test2 = np.concatenate([numerical_test, PCA_test_g, PCA_test_c], axis=1)


In [21]:
print(numerical_tr2.shape)
print(cat_tr.shape)

(21948, 1409)
(21948, 4)


### 4.4 FS by Variance Encoding<a class="anchor" id="4.4"></a>

[Back to Table of Contents](#0.1)

In [22]:
var_thresh = VarianceThreshold(VarianceThreshold_for_FS)
data = np.append(numerical_tr2,numerical_test2, axis=0)
data_transformed = var_thresh.fit_transform(data[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]

train_features = np.concatenate([cat_tr,train_features_transformed], axis=1)
test_features2 = np.concatenate([cat_test,test_features_transformed], axis=1)

In [23]:
print(train_features_transformed.shape)
print(train_features.shape)

(21948, 1012)
(21948, 1016)


In [24]:
train = pd.merge(pd.DataFrame(train_features),train_targets_scored, left_index=True, right_index=True)
test = pd.DataFrame(test_features2)
target = train[train_targets_scored.columns]
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [25]:
train

0    1     2    3         4         5         6         7         8  \
0      2.0  0.0  22.0  2.0 -0.286559 -1.011388 -1.357431 -0.041716  0.719019   
1      1.0  0.0  16.0  2.0  1.199685  0.691813  0.353695  0.558374 -0.497516   
2      0.0  0.0  20.0  0.0 -0.028694  1.490985  0.272541  0.359490  0.038331   
3      0.0  0.0   3.0  4.0  2.343522 -0.852713 -2.316440  0.301512 -0.150669   
4      1.0  1.0  12.0  3.0  1.447729 -0.863807 -0.346926 -0.227072 -1.026456   
...    ...  ...   ...  ...       ...       ...       ...       ...       ...   
21943  1.0  0.0  32.0  3.0 -0.365285  0.574150 -0.118761 -0.271980 -0.900899   
21944  2.0  1.0   6.0  3.0 -0.721089  0.924354  0.779755  0.509043 -0.582884   
21945  2.0  1.0  32.0  3.0 -1.057997  0.856044 -0.274794 -0.731116  0.670339   
21946  2.0  0.0   9.0  2.0  1.068126 -0.020533  0.084007  0.087675  0.570980   
21947  1.0  0.0  17.0  4.0 -0.554418 -2.097418 -1.625089  1.460639  2.245448   

              9        10        11        12        13        14        15  \
0     -0.298798  1.546151  0.155864  0.661714 -0.548724  0.295737 -1.071349   
1      0.845472 -1.243593 -0.590105 -0.181974  0.547967  0.165179  0.383917   
2      1.246733 -0.651185 -0.782581 -0.098601 -2.292704  0.888461 -0.558999   
3     -1.358486 -1.008581 -0.509950 -1.117638 -0.802450 -1.739990  1.385318   
4      0.858798 -0.344280 -0.715015 -1.216237  1.171604  0.333034  0.335085   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.019740  0.705188 -0.709982  0.485255  0.148203  1.119927 -0.327068   
21944 -0.031622  0.289149  0.182048 -0.657455 -1.500635  0.394979 -0.774565   
21945  0.654943 -0.301056  1.752037 -0.337616 -0.846707  1.178427  0.961687   
21946 -1.174608  0.736426  0.141364  1.724963  0.252759  0.798072 -2.229249   
21947  1.263222  2.093617  1.260040  1.894469  1.596927 -0.738421  1.182859   

             16        17        18        19        20        21        22  \
0     -1.120205  0.904383  0.363779 -0.523377 -0.698297 -1.369550 -1.695536   
1      0.413317  0.431386 -0.855132  0.489923  1.269330 -0.149157 -0.423792   
2      0.485378 -0.379193 -0.321293 -0.054187 -0.076751 -1.045070 -1.951027   
3     -0.209308 -1.029882  0.229093 -2.129180  2.080983 -1.390113 -1.126062   
4     -0.009513 -1.537557 -0.866434 -0.720440 -0.422244 -0.309671  1.018179   
...         ...       ...       ...       ...       ...       ...       ...   
21943  0.125266  0.308968  0.038492  0.220386 -0.625055 -0.779826  0.708461   
21944  0.242660  0.260730 -1.526452 -0.136004  1.167199 -0.535780  0.792831   
21945 -0.479190  0.216640  0.014218  0.181197 -0.265076 -1.154525  0.110929   
21946  0.012099  0.871252  0.419901  1.457526 -0.932893  1.393660  0.519332   
21947 -1.355048  0.061907 -1.783781 -0.081160 -0.228009 -0.995825 -1.880985   

             23        24        25        26        27        28        29  \
0      1.250671  0.570589  0.436027  0.290501  0.129888 -0.481883  0.761154   
1     -0.458679  0.402412 -0.555557  0.689364  0.186370 -0.709338 -0.208375   
2      0.533785  0.640179  0.735237 -1.349676  2.391083 -0.048482  1.576074   
3     -1.073452 -1.405924  0.034704 -1.023687  0.231648 -0.415427 -1.250653   
4      0.493778  1.709450  1.042281  1.140661 -0.607406 -1.509471  0.364082   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.701450  0.196888 -0.267468  0.469128  0.539585  0.125113  1.655753   
21944  0.534271 -0.702949 -0.566747 -1.043002 -0.416754 -1.043564  0.273571   
21945 -1.815658 -0.982967 -1.186046  0.449833  0.140572  0.189304  0.205938   
21946  2.454238  0.643132  1.021394  0.360162 -0.554870  1.296967 -1.775765   
21947 -1.312172 -1.202112 -0.693610 -1.224125 -0.482664  0.309355 -0.600661   

             30        31        32        33        34        35        36  \
0      0.419666 -1.109027 -0.059113 -0.409266 -0.235059  0.290278  0.383926   
1     -0.120770 -0.552012  1.673802 -0.3368

In [26]:
def process_data(data):
    data = pd.get_dummies(data, columns=[0,1,2,3])
    return data

### 4.5 CV folds<a class="anchor" id="4.5"></a>

[Back to Table of Contents](#0.1)

In [27]:
SEED = 62
NFOLDS = 5

# LOAD LIBRARIES (from PIP or Kaggle Dataset) 
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# LOAD FILES
scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
targets = scored.columns[1:]
scored = scored.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=SEED)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
scored['kfold'] = scored.drug_id.map(dct1)
scored.loc[scored.kfold.isna(),'kfold'] = scored.loc[scored.kfold.isna(),'sig_id'].map(dct2)
scored.kfold = scored.kfold.astype('int8')

folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

In [28]:
folds = train.copy()
folds = pd.merge(folds, scored[['sig_id','kfold']], how='left', left_on='sig_id', right_on='sig_id')

In [29]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1223)
(21948, 1224)
(3624, 1016)
(21948, 207)
(3982, 207)


### 4.6 Dataset Classes<a class="anchor" id="4.6"></a>

[Back to Table of Contents](#0.1)

In [30]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [31]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

### 4.7 Smoothing<a class="anchor" id="4.7"></a>

[Back to Table of Contents](#0.1)

In [32]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

### 4.8 Preprocessing<a class="anchor" id="4.8"></a>

[Back to Table of Contents](#0.1)

In [33]:
folds = process_data(folds)
test = process_data(test)
feature_cols = [c for c in folds.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

1057

## 5. Modeling<a class="anchor" id="5"></a>

[Back to Table of Contents](#0.1)

In [34]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25 #22 21
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
#NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1500

In [35]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(Dropout_Model)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(Dropout_Model)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [36]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    #train = process_data(folds)
    #test_ = process_data(test)
    train = folds
    test_ = test
    
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    '''
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            '''
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    #model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.load_state_dict(torch.load(f"../input/final-sub-ann-rankgauss-pca-training/FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


## 6. Prediction & Submission <a class="anchor" id="6"></a>

[Back to Table of Contents](#0.1)

In [37]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [38]:
# Averaging on multiple SEEDS

SEED = [62, 64, 93, 92, 19]
#SEED = [62]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions


In [39]:
test[target_cols].set_index(test_features.index)

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,re

In [40]:
len(target_cols)

206

In [41]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)    

CV log_loss:  0.12805215260573755


In [42]:
submission = sample_submission.drop(columns=target_cols).merge(test[target_cols].set_index(test_features.index), how='left', left_index=True, right_index=True).fillna(0)
submission.to_csv("submission.csv", index=False)